In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('../data/CoilData.csv')
coils = pd.read_csv('./output.csv')
coil_list = list(map(int,list(coils.columns)))
lst = []
for i in data['coil']:
    if i in coil_list:
        lst.append(1)
    else:
        lst.append(0)
data['contracted'] = lst
df = data

'''
Check how many times it happens that the coils are broken right after each other



'''
# df = pd.get_dummies(data, columns=['contracted'])

'\nCheck how many times it happens that the coils are broken right after each other\n\n\n\n'

In [3]:
# plt.figure(figsize=(20, 10))
# corr = df.corr()
# mask = np.zeros_like(corr)
# mask[np.triu_indices_from(mask)] = True
# heatmap = sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, annot=True)
# heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);
# plt.show()

In [4]:
'''
Fit a model for every type of metal --- 'analyse' column ---'''

"\nFit a model for every type of metal --- 'analyse' column ---"

## Data selection and partitioning

In [5]:
df = df.drop(columns=['coil','analyse'])

In [6]:
df['Thickness profile'] = df['Thickness profile'].apply(lambda x: x.replace('*******', ''))
df = df.replace('', np.nan, regex=True).dropna().astype(float)
df.head().T

,0,1,2,3,4
furnace Number,1.00,3.00,4.00,3.00,4.00
Hardness_1,10003.00,10123.00,10040.00,10243.00,10012.00
Hardness_2,101.00,101.00,102.00,102.00,100.00
Width,1302.10,1282.30,1297.40,1295.20,1293.30
Temperature before finishing mill,1147.00,1150.00,1183.00,1165.00,1192.00
Temperature after finishing mill,921.00,920.00,933.00,910.00,909.00
Thickness,4.36,4.37,4.43,4.44,3.95
Thickness profile,31.00,35.00,25.00,28.00,26.00
c,355.00,551.00,457.00,697.00,477.00
mn,2162.00,1985.00,1895.00,2008.00,1936.00


In [7]:
# making random samples for train and test sets, because the output.csv is ordered from highest 
# absolute difference to less ...
X = df.iloc[:,:-1]
y = df.contracted
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

'''
Balanced dataset for more realistic scores ,even 0 and 1 in train and test sets'''



## Baseline modeling

In [8]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression())
])

# Train model
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

# Make predictions
y_train_pred_proba = pipe.predict_proba(X_train)[:,1]
y_test_pred_proba = pipe.predict_proba(X_test)[:,1]

In [9]:
# Observe general model performance
print('train roc auc: {:.2f}'.format(roc_auc_score(y_train, y_train_pred_proba)))
print('test roc auc:   {:.2f}'.format(roc_auc_score(y_test, y_test_pred_proba)))

train roc auc: 0.85
test roc auc:   0.84


In [10]:
# Observe general model performance
y_train_pred = y_train_pred_proba > 0.5
y_test_pred = y_test_pred_proba > 0.5

print('train f1:      {:.2f}'.format(f1_score(y_train, y_train_pred)))
print('test f1:        {:.2f}'.format(f1_score(y_test, y_test_pred)))

train f1:      0.19
test f1:        0.18
